In [57]:
import pandas as pd
from collections import defaultdict
from itertools import combinations
import numpy as np
from statsmodels.stats.contingency_tables import mcnemar

In [37]:
qc = pd.read_csv("../../results/QC/illumina.qc.csv", index_col="run")

In [38]:
min_depth = 15
max_contam = 0.05
valid_samples = qc.query("coverage>=@min_depth and f_contam<=@max_contam").index

In [42]:
df = pd.read_csv("../../results/tables/sn_sp/classifications.illumina.csv")
df.query("run in @valid_samples", inplace=True)
df.set_index(["run", "drug", "tool"], drop=False, inplace=True)

In [43]:
df

,,,run,drug,classification,tool
run,drug,tool,,,,
ERR4798664,kanamycin,drprg,ERR4798664,kanamycin,TN,drprg
SRR6982346,kanamycin,drprg,SRR6982346,kanamycin,TN,drprg
ERR4796956,kanamycin,drprg,ERR4796956,kanamycin,TN,drprg
ERR4796656,kanamycin,drprg,ERR4796656,kanamycin,TN,drprg
ERR4829787,kanamycin,drprg,ERR4829787,kanamycin,TN,drprg
...,...,...,...,...,...,...
ERR2512943,rifampicin,tbprofiler,ERR2512943,rifampicin,TN,tbprofiler
ERR8700384,rifampicin,tbprofiler,ERR8700384,rifampicin,TN,tbprofiler
SRR6397652,rifampicin,tbprofiler,SRR6397652,rifampicin,TN,tbprofiler


In [41]:
DRUGS = set(df["drug"])
TOOLS = set(df["tool"])

In [55]:
M = defaultdict(dict)
for drug in DRUGS:
    d = {}
    for t in combinations(TOOLS, 2):
        sp = np.zeros((2, 2), dtype=int)
        sn = np.zeros((2, 2), dtype=int)
        t1, t2 = sorted(t)
        runs = df.query("tool==@t1 and drug==@drug")["run"]
        for r in runs:
            c1 = df.at[(r, drug, t1), "classification"]
            c2 = df.at[(r, drug, t2), "classification"]
            
            match (c1, c2):
                case ("TP", "TP"):  # sensitivity a
                    sn[0][0] += 1
                case ("TP", "FN"):  # sensitivity c
                    sn[1][0] += 1
                case ("FN", "TP"):  # sensitivity b
                    sn[0][1] += 1
                case ("FN", "FN"):  # sensitivity d
                    sn[1][1] += 1
                case ("TN", "TN"):  # specificity a
                    sp[0][0] += 1
                case ("TN", "FP"):  # specificity c
                    sp[1][0] += 1
                case ("FP", "TN"):  # specificity b
                    sp[0][1] += 1
                case ("FP", "FP"):  # specificity d
                    sp[1][1] += 1
                case _:
                    raise ValueError((c1, c2))

        d["sn"] = sn
        d["sp"] = sp
        M[(t1, t2)][drug] = d

In [79]:
for (t1, t2), d in M.items():
    for drug, mats in d.items():
        sn = mats["sn"]
        sp = mats["sp"]
        sn_res = mcnemar(sn, exact=True)
        
        print(f"T1: {t1}\tT2: {t2}\t{drug}\nSensitivity")
        print(sn_res)
        print("Specificity")
        
        sp_res = mcnemar(sp, exact=True)
        print(sp_res)

T1: drprg	T2: tbprofiler	capreomycin
Sensitivity
pvalue      1.0323214044661785e-21
statistic   1.0
Specificity
pvalue      2.837623469531536e-10
statistic   1.0
T1: drprg	T2: tbprofiler	moxifloxacin
Sensitivity
pvalue      2.9802322387695312e-08
statistic   1.0
Specificity
pvalue      0.00031255115754902363
statistic   7.0
T1: drprg	T2: tbprofiler	linezolid
Sensitivity
pvalue      1.0
statistic   0.0
Specificity
pvalue      1.0
statistic   0.0
T1: drprg	T2: tbprofiler	ethambutol
Sensitivity
pvalue      2.744048833847046e-05
statistic   3.0
Specificity
pvalue      0.0005350527353584766
statistic   6.0
T1: drprg	T2: tbprofiler	pyrazinamide
Sensitivity
pvalue      0.08380154521018608
statistic   84.0
Specificity
pvalue      3.1798876310461662e-06
statistic   27.0
T1: drprg	T2: tbprofiler	amikacin
Sensitivity
pvalue      6.462348535570529e-27
statistic   0.0
Specificity
pvalue      1.5366822481155396e-08
statistic   1.0
T1: drprg	T2: tbprofiler	isoniazid
Sensitivity
pvalue      1.99201957

In [61]:
dir(sn_res)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'pvalue',
 'statistic']

In [71]:
sn_res.pvalue

1.478092302309394e-18

In [72]:
sn_res.statistic

77.28735632183908

In [73]:
t1

'drprg'

In [74]:
t2

'mykrobe'

In [75]:
drug

'kanamycin'

In [76]:
sp_res.pvalue

5.103985260291326e-05

In [77]:
sp_res.statistic

16.40909090909091